In [1]:
import pandas as pd
import numpy as np
import datetime
import time 
import os
import gc
import re
import sys
from functools import partial

from utils import ignore_warnings, load_data

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# ignore_warnings()

In [3]:
%%time
# nrows = 10000
nrows = None
train = load_data('train', nrows=nrows)#, verbose=True)

CPU times: user 18.3 s, sys: 1.39 s, total: 19.7 s
Wall time: 19.7 s


In [97]:
a =load_data('train', nrows=nrows).impressions.dropna().str.split('|')

In [98]:
Is = list(set([j for i in a for j in i]))

In [99]:
Is = [int(i) for i in Is]

In [100]:
m = load_data('item_metadata')
# m = meta_features

In [101]:
ms = m.index.unique()

In [102]:
len(Is)

853540

In [103]:
len(ms)

927142

In [104]:
len(set(Is)-set(ms))

740203

In [96]:
len(set(Is)-set(ms))

853040

In [4]:
# test = load_data('test', nrows=nrows)

In [5]:
meta = load_data('item_metadata', verbose=True)

Memory usage: 295.55 mb


### convert all string type to lower case

In [6]:
%%time
cols_lower = ['action_type', 'reference', 'platform', 'city', 'device', 'current_filters']
for c in cols_lower:
    print('lowering ', c)
    train[c] = train[c].str.lower()

meta['properties'] = meta['properties'].str.lower()


lowering  action_type
lowering  reference
lowering  platform
lowering  city
lowering  device
lowering  current_filters
CPU times: user 12.9 s, sys: 816 ms, total: 13.7 s
Wall time: 13.7 s


### reduce train df size

In [7]:
%%time
create_mapper = lambda unique_values: {v: k for k, v in enumerate(unique_values)}
def convert(df, col, mapper):
    df.loc[df[col].notna(), col] = df.loc[df[col].notna()][col].map(mapper)

def replace(str_list, replace_dict):
    return '|'.join([replace_dict[s] for s in str_list])

# reduce all data types to int and cache the mapping used
t_int = time.time()
fprint = lambda msg: print(f"{msg:<40} {'='*20} time elapsed = {(time.time()-t_int)/60:.2f} mins")
# ===========================================================================================
# 0) all session_ids and user_ids
unique_user_ids = train.user_id.unique()
user_id_mapper = create_mapper(unique_user_ids)
convert(train, 'user_id', user_id_mapper)
fprint('done user_id')
                           
unique_session_ids = train.session_id.unique()
session_id_mapper = create_mapper(unique_session_ids)
convert(train, 'session_id', session_id_mapper)
fprint('done session_id')
# ===========================================================================================
# 1) get timestamp range and subtract the min
min_ts = train['timestamp'].min()
train['timestamp'] -= min_ts
fprint('done timestamp')
# ===========================================================================================
# 2) action_type
unique_action_types = train.action_type.dropna().unique()
action_mapper = create_mapper(unique_action_types)
convert(train, 'action_type', action_mapper)
fprint('done action_type')
# ===========================================================================================
# # 3) all item_ids and reference
impression_lists = train[train.impressions.notna()].impressions.str.split('|')
unique_impressions = list(set([j for i in impression_lists for j in i]))
meta_ids = list(meta['item_id'].astype(str).unique())
complete_item_ids = (list(train[train.reference.notna()].reference.unique()) 
                     + unique_impressions
                     + meta_ids)
reference_item_id_mapper = create_mapper(complete_item_ids)
reference_item_id_mapper_str = {k: str(v) for k, v in reference_item_id_mapper.items()}
reference_item_id_mapper_int = {int(k): v for k,v in reference_item_id_mapper.items() 
                                if not re.search('[a-zA-Z]', k)}
convert(train, 'reference', reference_item_id_mapper)
fprint('done reference')

replace_impression = partial(replace, replace_dict=reference_item_id_mapper_str)
train['imps'] = train['impressions'].str.split('|')
train.loc[train['impressions'].notna(), 'imps'] = train[train['impressions'].notna()]['imps'].apply(replace_impression)
del train['impressions']  
fprint('done impressions item_id for train')
convert(meta, 'item_id', reference_item_id_mapper_int)
fprint('done item_id for meta')

# ===========================================================================================
# 4) all platform
unique_platform = train.platform.dropna().unique()
platform_mapper = create_mapper(unique_platform)
convert(train, 'platform', platform_mapper)
fprint('done platform')
# ===========================================================================================

# 4) all cities
unique_cities = train.city.dropna().unique()
city_mapper = create_mapper(unique_cities)
convert(train, 'city', city_mapper)
fprint('done city')
# ===========================================================================================

# 5) all device
unique_device = train.device.dropna().unique()
device_mapper = create_mapper(unique_device)
convert(train, 'device', device_mapper)
fprint('done device')
# ===========================================================================================

# 6) filters/properties
# unique filters from filters
filter_lists = train[train.current_filters.notna()].current_filters.str.split('|')
unique_filters = list(set([j for i in filter_lists for j in i]))
# unique properties from meta
properties_lists = meta.properties.str.split('|')
unique_properties = list(set([j for i in properties_lists for j in i]))
all_properties = list(set(unique_filters + unique_properties))
properties_mapper = create_mapper(all_properties)
properties_mapper_str = {k: str(v) for k, v in properties_mapper.items()}
                    
replace_properties = partial(replace, replace_dict=properties_mapper_str)
train['cf'] = train['current_filters'].str.split('|')
train.loc[train['cf'].notna(), 'cf'] = train[train['cf'].notna()]['cf'].apply(replace_properties)
del train['current_filters']
fprint('done cf')

meta['ps_list'] = meta.properties.str.split('|')
meta['properties_int'] = meta.ps_list.apply(replace_properties)
del meta['ps_list']#, meta['properties']
fprint('done meta properties')
                           
# rename
train.rename(columns={'imps': 'impressions', 'cf': 'current_filters'}, inplace=True)
# meta.rename(columns={'ps': 'properties'}, inplace=True)

done user_id                             ==================== time elapsed = 0.12 mins
done session_id                          ==================== time elapsed = 0.23 mins
done timestamp                           ==================== time elapsed = 0.24 mins
done action_type                         ==================== time elapsed = 0.35 mins
done reference                           ==================== time elapsed = 0.73 mins
done impressions item_id for train       ==================== time elapsed = 1.02 mins
done item_id for meta                    ==================== time elapsed = 1.06 mins
done platform                            ==================== time elapsed = 1.15 mins
done city                                ==================== time elapsed = 1.27 mins
done device                              ==================== time elapsed = 1.36 mins
done cf                                  ==================== time elapsed = 1.54 mins
done meta properties                     ==

In [8]:
meta.head(2)

,item_id,properties,properties_int
0,1253808,satellite tv|golf course|airport shuttle|cosme...,0|171|7|55|17|140|99|94|181|149|51|177|22|50|1...
1,1253809,satellite tv|cosmetic mirror|safe (hotel)|tele...,0|55|17|140|99|94|181|124|51|22|164|41|180|150...


In [9]:
# save all the mappings
mapper_dict = {}
mapper_dict['user_id'] = user_id_mapper
mapper_dict['session_id'] = session_id_mapper
mapper_dict['action_type'] = action_mapper
mapper_dict['reference_item'] = reference_item_id_mapper
mapper_dict['user_id'] = user_id_mapper
mapper_dict['platfrom'] = platform_mapper
mapper_dict['city'] = city_mapper
mapper_dict['device'] = device_mapper
mapper_dict['properties'] = properties_mapper

In [10]:
# train.memory_usage(deep=True).sum() / 1024 ** 2

In [11]:
train.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,prices,impressions,current_filters
0,0,0,7052,1,0,0,0,0,0,NaN,NaN,NaN
1,0,0,7114,2,1,1676901,0,0,0,NaN,NaN,NaN
2,0,0,7114,3,1,1676901,0,0,0,NaN,NaN,NaN
3,0,0,7124,4,1,1676901,0,0,0,NaN,NaN,NaN
4,0,0,7124,5,1,1500794,0,0,0,NaN,NaN,NaN


### preprocessing

In [12]:
mapper_dict['action_type']

{'search for poi': 0,
 'interaction item image': 1,
 'clickout item': 2,
 'interaction item info': 3,
 'interaction item deals': 4,
 'search for destination': 5,
 'filter selection': 6,
 'interaction item rating': 7,
 'search for item': 8,
 'change of sort order': 9}

In [13]:
df = train.iloc[:10000].copy()

In [14]:
df.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,prices,impressions,current_filters
0,0,0,7052,1,0,0,0,0,0,NaN,NaN,NaN
1,0,0,7114,2,1,1676901,0,0,0,NaN,NaN,NaN
2,0,0,7114,3,1,1676901,0,0,0,NaN,NaN,NaN
3,0,0,7124,4,1,1676901,0,0,0,NaN,NaN,NaN
4,0,0,7124,5,1,1500794,0,0,0,NaN,NaN,NaN


In [15]:
data_source = 'train'
# 1) Cliping sessions up to last clickout (if there is clickout)
def clip_last_click(grp):
    # 'clickout item': 2
    check = grp.action_type.values == 2
    if check.sum() != 0:
        return grp.iloc[:np.argwhere(check)[-1][0]+1]
    else:
        return grp

fprint('cliping sessions off up to last clickout')
df = df.groupby('session_id').apply(clip_last_click).reset_index(drop=True)

cliping sessions off up to last clickout ==================== time elapsed = 1.63 mins


In [16]:
df.shape

(8462, 12)

In [17]:
sids = df.session_id.unique()
df[df.session_id==np.random.choice(sids, 1)[0]]

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,prices,impressions,current_filters
4362,292,293,64567,1,1,1472812,13,281,0,NaN,NaN,NaN
4363,292,293,64567,2,1,1472812,13,281,0,NaN,NaN,NaN
4364,292,293,64577,3,1,1472812,13,281,0,NaN,NaN,NaN
4365,292,293,64577,4,1,1472812,13,281,0,NaN,NaN,NaN
4366,292,293,64577,5,1,1472812,13,281,0,NaN,NaN,NaN
4367,292,293,64577,6,1,1472812,13,281,0,NaN,NaN,NaN
4368,292,293,64588,7,1,1472812,13,281,0,NaN,NaN,NaN
4369,292,293,64588,8,1,1472812,13,281,0,NaN,NaN,NaN
4370,292,293,64627,9,9,28,13,281,0,NaN,NaN,NaN
4371,292,293,64640,10,9,70,13,281,0,NaN,NaN,NaN


In [18]:
# 2) Only select sessions that have a click out
def filter_clickout(grp, mode):
    # sessions has clickouts
    # 'clickout item': 2
    has_clickout = (grp['action_type'].values == 2).sum() != 0
    if mode == 'train':
        # last row has reference and it's not nan
        has_ref = ((grp['action_type'].iloc[-1] == 2) &
                   (grp.iloc[-1][['impressions', 'reference', 'prices']].isna().sum() == 0))
    else:
        # test should have the last reference as nan for clickout
        has_ref = ((grp['action_type'].iloc[-1] == 2) &
                   (grp.iloc[-1][['reference']].isna()))
    return has_clickout & has_ref

fprint('filtering out sessions without clickout and reference, or clickout is not valid')
fprint(f'length before filtering: {len(df):,}')
filter_clickout_ = partial(filter_clickout, mode=data_source)
valid_clicked = df.groupby('session_id').apply(filter_clickout_)
click_session_ids = valid_clicked[valid_clicked].index
# filter
df = df[df.session_id.isin(click_session_ids)].reset_index(drop=True)
# del valid_clicked, click_session_ids
gc.collect()
fprint(f'{data_source} length after filtering: {len(df):,}')

filtering out sessions without clickout and reference, or clickout is not valid ==================== time elapsed = 1.63 mins
length before filtering: 8,462           ==================== time elapsed = 1.63 mins
train length after filtering: 7,432      ==================== time elapsed = 1.66 mins


In [19]:
df.shape

(7432, 12)

### create session fts

In [20]:
# some custom funcs used in agggregation
def mean_dwell_time(x):
    if len(x) == 1:
        return np.nan
    else:
        return np.mean(np.diff(np.sort(x)))
    
def median_dwell_time(x):
    if len(x) == 1:
        return np.nan
    else:
        return np.median(np.diff(np.sort(x)))

def dwell_time_before_last(x):
    if len(x) == 1:
        return np.nan
    else:
        sorted_x = np.sort(x)
        return sorted_x[-1] - sorted_x[-2]
    
def var_dwell_time(x):
    if len(x) == 1:
        return np.nan
    else:
        return np.var(np.diff(np.sort(x)))

def get_first(x):
    return x.iloc[0]

def get_last(x):
    return x.iloc[-1]

def n_clickouts(x):
    # 'clickout item': 2
    return (x.values == 2).sum()

def click_rel_pos_avg(x):
    # 'clickout item': 2
    return np.mean(np.argwhere((x.values == 2))+1)/len(x)

def span(x):
    return x.max() - x.min()

def second_last(x):
    if len(x) == 1:
        return np.nan
    else:
        return x.iloc[-2]
    
# define some aggs
session_aggs = {'timestamp': [span, mean_dwell_time, var_dwell_time, median_dwell_time, dwell_time_before_last],
                'step': ['max'],
                'action_type': ['nunique', n_clickouts, click_rel_pos_avg, second_last],
                'reference': [second_last],
                'city': ['nunique', get_last],
                'platform': [get_last],
                'device': [get_last],
                'n_imps': [get_last],
                'n_filters': [get_last],
                }

df['imp_list'] = df.impressions.str.split('|')
df['n_imps'] = df.imp_list.str.len()
del df['imp_list']
gc.collect()
df['cf_list'] = df.current_filters.str.split('|')
df['n_filters'] = df.cf_list.str.len()
del df['cf_list']
gc.collect()

session_grp = df.groupby('session_id')
session_fts = session_grp.agg(session_aggs)
session_fts.columns = ['_'.join(col).strip() for col in session_fts.columns.values]
del df['n_imps'], df['n_filters']
gc.collect()

# fprint('working on n_match')
# def n_matches(df):
#     last_row = df.iloc[-1]
#     if pd.isna(last_row.current_filters):
#         return np.nan
#     else:
#         cf = last_row.current_filters.split('|')
#         imp = last_row.impressions.split('|')
#         return len(set(cf).intersection(imp))
# n_matches = session_grp.apply(n_matches, axis=1)
# fprint('done n_match')

63

In [21]:
session_fts.head()

,timestamp_span,timestamp_mean_dwell_time,timestamp_var_dwell_time,timestamp_median_dwell_time,timestamp_dwell_time_before_last,step_max,action_type_nunique,action_type_n_clickouts,action_type_click_rel_pos_avg,action_type_second_last,reference_second_last,city_nunique,city_get_last,platform_get_last,device_get_last,n_imps_get_last,n_filters_get_last
session_id,,,,,,,,,,,,,,,,,
0,1025,68.333333,52782.355556,0.0,16.0,16,3,2,0.937500,0.0,3.0,1,0,0,0,25.0,NaN
2,134,3.941176,61.702422,0.0,1.0,35,3,1,1.000000,3.0,2037703.0,1,2,2,0,25.0,NaN
3,614,102.333333,12221.555556,77.5,330.0,7,4,2,0.928571,2.0,2158255.0,1,3,3,1,25.0,NaN
4,253,4.685185,167.512003,0.0,38.0,55,4,2,0.990909,2.0,1291523.0,1,4,4,1,25.0,8.0
5,79,39.500000,1056.250000,39.5,72.0,3,3,1,1.000000,8.0,1607671.0,2,6,0,1,25.0,NaN


### create meta features

In [22]:
mapper_dict.keys()

dict_keys(['user_id', 'session_id', 'action_type', 'reference_item', 'platfrom', 'city', 'device', 'properties'])

In [23]:
meta.shape

(927142, 3)

In [31]:
# def create_rating_colum(meta):
#     # create rating columns
#     ratings = ['good rating', 'satisfactory rating', 'excellent rating']
#     meta_mapper = mapper_dict['properties']
#     ratings_id = [str(meta_mapper[v]) for v in ratings]
#     for k, v in enumerate(ratings_id):
#         meta[ratings[k].replace(' ', '_')] = meta['properties'].str.findall(f'(\||\b){v}').str.len()
def create_rating_colum(meta):
    # create rating columns
    ratings = ['good rating', 'satisfactory rating', 'excellent rating']
    for r in ratings:
        meta[r.replace(' ', '_')] = meta['properties'].str.findall(f'(\||\b){r}').str.len()

In [32]:
def create_meta_fts(meta, df, recompute=False):
    meta_file = f'./data/meta_fts.csv'
    if os.path.isfile(meta_file) and not recompute:
        print(f'{meta_file} exists, reload')
        meta = pd.read_csv(meta_file)
        # convert 'list' to list
        meta.loc[:, 'ps'] = meta.loc[:, 'ps'].apply(literal_eval)
        meta['ps'] = meta.ps.apply(lambda x: [int(i) for i in x])
        meta = meta.set_index('item_id')
    else:
        # add ratings
        create_rating_colum(meta)
        
        meta['ps'] = meta['properties_int'].str.split('|')
#         meta['ps'] = meta['ps'].apply(lambda x: [int(i) for i in x])
        # numer of properties
        meta['nprop'] = meta['ps'].str.len()
        # star ratings
        meta['star'] = meta['properties'].str.extract('[\|](\d) star')
        meta['star'] = meta['star'].astype(float)
        del meta['properties']
        meta.rename(columns={'properties_int': 'properties'}, inplace=True)
        
        # action encodings
        action_encodings = action_encoding(df)
        meta = pd.merge(meta, action_encodings, left_on='item_id', right_on='reference')
        # choose columns
        act_cols = [c for c in action_encodings.columns if c != 'reference']

        use_cols = ['item_id', 'nprop', 'star', 'good_rating', 'satisfactory_rating',
                    'excellent_rating', 'properties']
        use_cols += act_cols
        meta = meta[use_cols].set_index('item_id')
        meta.to_csv(meta_file)

    return meta

In [33]:
def action_encoding(df):
    # now group on reference ids
    action_grp = df.groupby('reference')['action_type']
    # get value counts for each action type
    action_ctn = action_grp.value_counts()
    action_ctn_df = action_ctn.reset_index(name='ctn')

    # list of all unique action type
    # actions = list(np.sort(df.action_type.unique()))
    action_mapper = mapper_dict['action_type']
    # actions_id = action_mapper.values()
    actions_name = list(action_mapper.keys())

    # create ohe
    ohe = pd.DataFrame(np.eye(len(actions_name), dtype=int)[action_ctn_df.action_type.values], 
                       columns=actions_name)
    ohe = ohe.mul(action_ctn_df['ctn'], axis=0)
    action_ctn_df = pd.concat([action_ctn_df, ohe], axis=1)

    action_encoding = action_ctn_df.groupby('reference')[actions_name].sum()
    # also add normalized percentage over count of each actions over total
    normalized = action_encoding.div(action_encoding.sum(axis=1)+1, axis=0) # +1 for smoothing avoiding leakage
    action_encoding = action_encoding.join(normalized, lsuffix='_ctn', rsuffix='_per')
    # set the popularity (i.e the number of clickout) counts to rank, avoid leakage
    # but even if we convert them to rank it still leaks (e.g. 0 clicks out will always stay behind a rank threshold)
    # so for now we drop it (try to use embeddings)
    del action_encoding['clickout item_ctn']
    return action_encoding.reset_index()

In [34]:
meta.head()

,item_id,properties,properties_int
0,1253808,satellite tv|golf course|airport shuttle|cosme...,0|171|7|55|17|140|99|94|181|149|51|177|22|50|1...
1,1253809,satellite tv|cosmetic mirror|safe (hotel)|tele...,0|55|17|140|99|94|181|124|51|22|164|41|180|150...
2,1253810,satellite tv|cosmetic mirror|safe (hotel)|tele...,0|55|17|140|99|181|41|168|180|150|115|6|111|14...
3,1253811,satellite tv|sailing|cosmetic mirror|telephone...,0|43|55|140|99|177|22|164|41|28|180|150|169|11...
4,1253812,satellite tv|sailing|diving|cosmetic mirror|sa...,0|43|65|55|17|140|99|94|181|124|51|177|16|22|5...


In [35]:
%%time
meta_features = create_meta_fts(meta, train, recompute=True)

CPU times: user 26.5 s, sys: 353 ms, total: 26.8 s
Wall time: 26.9 s


In [146]:
# ====================================# ====================================# ====================================
def get_session_item_pairs(args):
    # grab the args
    gids, session_df, meta_df, data_source = args
    # selecting the assigned session ids and grouping on session level
    grps = (session_df[session_df['session_id'].isin(gids)]
            .reset_index(drop=True)
            .groupby('session_id'))

#      meta_df= create_meta_fts(data_source)
    # use apply to compute session level features
    session_compute_func = partial(compute_session_item_pair, meta_df=meta_df, data_source=data_source)
    session_features = grps.apply(session_compute_func)

    return session_features

ppp = {'v':0}
def pp():
    pass
#     print(ppp['v'])
#     ppp['v']+=1
    
actions_natural = mapper_dict['action_type']
def compute_session_item_pair(session_df, meta_df, data_source):
    sdf = session_df.copy()
    # get last row and the rows above
    last_row = sdf.iloc[-1]
    above = sdf.iloc[:-1]
    pp()
    # get previous appeared impressions
    prev = above[above['impressions'].notnull()]
    prev_imps = prev['impressions'].str.split('|')
    imps = [j for i in prev_imps for j in i]
    prev_imps_ctn = pd.value_counts(imps).to_dict()
    pp()
    # get previous appeared clickout item_id counts
    prev_cos = prev['reference'].values
    prev_cos_ctn = pd.value_counts(prev_cos).to_dict()
    pp()
    # get last row
    imp_l = last_row['impressions'].split('|')
    prices = last_row['prices'].split('|')
    prices = [int(p) for p in prices]
    pp()
    # how many times the item_id has appeared before
    appeared = [int(prev_imps_ctn[i]) if i in imps else 0 for i in imp_l]
    pp()
    # how many times the item_id was clicked before
    appeared_co = [int(prev_cos_ctn[int(i)]) if int(i) in prev_cos else 0 for i in imp_l]
    pp()
    # the location of the impression
    locs = list(range(len(imp_l)))

    # previous occured action_types
    actions_above = above['action_type'].dropna().values
    actions_above_ohe = np.eye(10, dtype=int)[actions_above].sum(axis=0)

    # build the df
    result = pd.DataFrame({'appeared': appeared, 'appeared_co': appeared_co, 'location': locs, 'price': prices},
                          index=imp_l)
    # keep the time for split 
    result['ts'] = last_row['timestamp']
    result.index.name = 'item_id'
    price_ind = np.argsort(result['price'].values) + 1
    result['rel_price_rank'] = price_ind / len(imp_l)

    result['price_mean'] = np.mean(result['price'])
    result['price_median'] = np.median(result['price'])

    result_price = result['price'].values
    result_price_mean = result['price_mean'].values
    result_price_median = result['price_median'].values

    result['diff_mean'] = result_price - result_price_mean
    result['diff_median'] = result_price - result_price_median
    result['diff_mean_rel'] = (result_price - result_price_mean) / result_price
    result['diff_median_rel'] = (result_price - result_price_median) / result_price

    # add previous action type cols (same for all rows)
    prev_act_cols = [f'prev_{i}' for i in actions_natural.keys()]
    result[prev_act_cols] = pd.DataFrame(np.tile(actions_above_ohe, (len(result), 1)), index=result.index)

    # fetch the meta data
    result.index = result.index.astype(int)
    result = result.join(meta_df, on='item_id')
#     result['p_mean'] = np.mean(result['n_clicks'].values)
    result['star_mean'] = np.mean(result['star'].values)
    result['gr_mean'] = np.mean(result['good_rating'].values)
    result['sr_mean'] = np.mean(result['satisfactory_rating'].values)
    result['er_mean'] = np.mean(result['excellent_rating'].values)

    # add number of matched filters
    cfilter = last_row['current_filters']
    mfilter = result['properties']
    if pd.notna(cfilter):# and pd.notna(mfilter):
        cfilters = cfilter.split('|')
        mfilters = mfilter.str.split('|')
        result['n_matches'] = [len(set(cfilters).intersection(p)) if type(p)!=float else np.nan for p in mfilters]
        result['n_matches_per'] = result['n_matches']/len(cfilter)
    else:
        result['n_matches'] = np.nan
    # reset index
    result.reset_index(inplace=True)

    if data_source == 'train':
        # get target
        ref = int(last_row['reference'])
        result['target'] = (result['item_id'].values == ref).astype(int)
    return result


# def generate_session_item_pairs(data_source, sessions_df, meta_df, nprocs=None):
def generate_session_item_pairs(data_source, sessions_df, meta_df, nprocs=None):
    t1 = time.time()
    if nprocs is None:
        nprocs = mp.cpu_count() - 1
        nprocs = 2
        print('Using {} cores'.format(nprocs))
    
    # get all the  
    sids = sessions_df['session_id'].unique()

    pairs = []
    
    # create iterator to pass in args
    def args_gen():
        for i in range(nprocs):
            yield (sids[range(i, len(sids), nprocs)], sessions_df, meta_df, data_source)

    # init multiprocessing pool
    pool = mp.Pool(nprocs)
    for pair in pool.map(get_session_item_pairs, args_gen()):
        pairs.append(pair)
    pool.close()
    pool.join()
    print('Done genearting, total time took: {0:.2f}mins'.format((time.time() - t1) / 60))

    return pd.concat(pairs, axis=0)


In [147]:
import multiprocessing as mp
session_items = generate_session_item_pairs(data_source, df, meta_features)

Using 2 cores
Done genearting, total time took: 0.16mins


In [148]:
# join
session_items.reset_index(level='session_id', inplace=True)
session_items.set_index('session_id', inplace=True)

In [149]:
session_items.head()

,item_id,appeared,appeared_co,location,price,ts,rel_price_rank,price_mean,price_median,diff_mean,...,interaction item rating_per,search for item_per,change of sort order_per,star_mean,gr_mean,sr_mean,er_mean,n_matches,target,n_matches_per
session_id,,,,,,,,,,,,,,,,,,,,,
0,2171750,0,0,0,162,8077,0.36,110.04,118.0,51.96,...,0.008065,0.004839,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN
0,1911504,1,0,1,25,8077,0.08,110.04,118.0,-85.04,...,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN
0,1897502,0,0,2,150,8077,0.64,110.04,118.0,39.96,...,0.007828,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN
0,1427502,0,0,3,143,8077,0.24,110.04,118.0,32.96,...,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN
0,1556460,0,0,4,101,8077,0.52,110.04,118.0,-9.04,...,0.013245,0.003311,0.0,NaN,NaN,NaN,NaN,NaN,0,NaN


In [150]:
final = session_items.join(session_fts)

In [ ]:
final.to_hdf